In [ ]:
%pip install transformers
%pip install accelerate
%pip install safetensors
%pip install invisible-watermark>=0.2.0
%pip install diffusers --upgrade

NORMAL INSTALL

In [ ]:
!git clone https://github.com/baegwangbin/surface_normal_uncertainty

In [ ]:
%cd surface_normal_uncertainty

In [ ]:
%pip install -r requirements.txt

In [ ]:
# Import necessary libraries and modules
from PIL import Image
from transformers import pipeline
import numpy as np
import cv2
import torch
from diffusers.utils import load_image

# File path variable
image_file_path = "/notebooks/threestudio/load/images/dragon3_rgba.png"  # specify your image file path here
# Remove '_rgba' from file path
image_file_path_base = image_file_path.replace('_rgba', '')

# Load image
image = load_image(image_file_path).convert("RGB")

depth_estimator = pipeline('depth-estimation')
# Depth Estimation
image = depth_estimator(image)['depth']
image = np.array(image)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
image = Image.fromarray(image)
# Save depth map
image.save(image_file_path_base[:-4] + '_depth.png')
image.show()
## NORMAL
image = load_image(image_file_path)
depth_estimator = pipeline("depth-estimation", model ="Intel/dpt-large" )

image = depth_estimator(image)['predicted_depth'][0]

image = image.numpy()
temp = Image.fromarray(image)

temp.show()

image_depth = image.copy()
image_depth -= np.min(image_depth)
image_depth /= np.max(image_depth)

bg_threhold = 0.1

x = cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=3)
x[image_depth < bg_threhold] = 0

y = cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=3)
y[image_depth < bg_threhold] = 0

z = np.ones_like(x) * np.pi * 2.0

image = np.stack([x, y, z], axis=2)
image /= np.sum(image ** 2.0, axis=2, keepdims=True) ** 0.5
image = (image * 127.5 + 127.5).clip(0, 255).astype(np.uint8)
image = Image.fromarray(image)

image.save(image_file_path_base[:-4] + '_normal.png')
image.show()